In [ ]:
import os
import shutil
import glob
import yaml
import mlflow
import random
from ultralytics import YOLO
from mlflow.tracking import MlflowClient

# ==========================================
# 1. CONFIGURACIÓN DE RUTAS (DINÁMICAS)
# ==========================================
cwd = os.getcwd()
BASE_DIR = os.path.abspath(os.path.join(cwd, "..")) if os.path.basename(cwd).lower() in ["notebooks", "notebook"] else cwd

# Rutas de datos
ORIGINAL_DATA = os.path.join(BASE_DIR, "data", "base_dataset", "Living-Room-9")
CLEAN_DATASET = os.path.join(BASE_DIR, "data", "processed_dataset")
MODELS_DIR = os.path.join(BASE_DIR, "models")
MODELS_HISTORY = os.path.join(BASE_DIR, "models_history")
MLFLOW_DB = os.path.join(BASE_DIR, "mlflow.db")

# Configuración de modelo
REGISTERED_MODEL_NAME = "Furniture_Model_YOLO"
# Mapeo: ID Original -> Nuestro ID (0:Sofa, 1:Rug, 2:Pillows)
ID_MAP = {12: 0, 11: 1, 19: 2}

# Crear directorios necesarios
for d in [MODELS_DIR, MODELS_HISTORY, CLEAN_DATASET]:
    os.makedirs(d, exist_ok=True)

# Configurar MLflow con ruta absoluta
db_uri = f"sqlite:///{MLFLOW_DB.replace(os.sep, '/')}"
mlflow.set_tracking_uri(db_uri)
mlflow.set_experiment("Furniture_Detection_System")

# ==========================================
# 2. RECONSTRUCCIÓN Y SANITIZACIÓN DEL DATASET
# ==========================================
def build_clean_dataset():
    """
    Crea una copia limpia del dataset traduciendo IDs y eliminando imágenes sin etiquetas.
    Esto evita el error de '0 instances' y 'background images' en YOLO.
    """
    print(f" Limpiando y reconstruyendo dataset en: {CLEAN_DATASET}")
    
    # Limpiar si ya existe para forzar frescura
    if os.path.exists(CLEAN_DATASET):
        shutil.rmtree(CLEAN_DATASET)

    for split in ["train", "valid"]:
        img_dst = os.makedirs(os.path.join(CLEAN_DATASET, split, "images"), exist_ok=True)
        lbl_dst = os.makedirs(os.path.join(CLEAN_DATASET, split, "labels"), exist_ok=True)
        
        src_imgs = glob.glob(os.path.join(ORIGINAL_DATA, split, "images", "*"))
        count = 0
        
        for img_path in src_imgs:
            name = os.path.splitext(os.path.basename(img_path))[0]
            label_src = os.path.join(ORIGINAL_DATA, split, "labels", name + ".txt")
            
            if os.path.exists(label_src):
                with open(label_src, 'r') as f:
                    lines = f.readlines()
                
                # Traducir IDs y filtrar solo Sofa, Rug y Pillows
                new_lines = []
                for line in lines:
                    parts = line.split()
                    if parts and int(parts[0]) in ID_MAP:
                        new_lines.append(f"{ID_MAP[int(parts[0])]} {' '.join(parts[1:])}\n")
                
                # Solo copiamos si la imagen contiene al menos un objeto de interés
                if new_lines:
                    shutil.copy(img_path, os.path.join(CLEAN_DATASET, split, "images"))
                    with open(os.path.join(CLEAN_DATASET, split, "labels", name + ".txt"), 'w') as f:
                        f.writelines(new_lines)
                    count += 1
        print(f" {split}: {count} imágenes procesadas con éxito.")

build_clean_dataset()

# ==========================================
# 3. GENERACIÓN DE CONFIGURACIÓN YAML
# ==========================================
# Usamos rutas absolutas para que YOLO no falle independientemente de dónde se ejecute
yaml_path = os.path.join(BASE_DIR, "data", "v1_fixed_config.yaml")
yaml_data = {
    'path': CLEAN_DATASET.replace('\\', '/'),
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 3,
    'names': {0: 'Sofa', 1: 'Rug', 2: 'Pillows'}
}

with open(yaml_path, 'w') as f:
    yaml.dump(yaml_data, f)

# ==========================================
# 4. ENTRENAMIENTO v1 (BASELINE)
# ==========================================
def train_v1():
    print("\n🚀 Iniciando entrenamiento del Baseline v1...")
    if mlflow.active_run(): mlflow.end_run()

    model = YOLO('yolov8n.pt')
    
    with mlflow.start_run(run_name="v1_Initial_Baseline") as run:
        results = model.train(
            data=yaml_path,
            epochs=5,
            imgsz=480,
            batch=16,
            project=MODELS_HISTORY,
            name="v1_run",
            exist_ok=True,
            fraction=0.05,
            freeze=10,
            overlap_mask=False,
            val=True,
        )
        
        # Registro de métricas
        map50 = results.box.map50
        mlflow.log_metric("map50", map50)
        
        # Guardar pesos finales
        src_weights = os.path.join(MODELS_HISTORY, "v1_run", "weights", "best.pt")
        dst_weights = os.path.join(MODELS_DIR, "best_v1.pt")
        shutil.copy(src_weights, dst_weights)
        
        # Registro en el Model Registry de MLflow
        client = MlflowClient()
        try:
            client.create_registered_model(REGISTERED_MODEL_NAME)
        except:
            pass
        
        mv = client.create_model_version(
            name=REGISTERED_MODEL_NAME,
            source=f"runs:/{run.info.run_id}/weights",
            run_id=run.info.run_id
        )
        print(f"🏁 v1 Registrada exitosamente. mAP: {map50:.4f}")

if __name__ == "__main__":
    train_v1()

c:\Users\andre\OneDrive\Documents\UPS\Inteligencia Artificial\YOLOv8_living-room_furniture\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026/02/01 09:25:19 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/01 09:25:19 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/01 09:25:19 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/01 09:25:19 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/01 09:25:19 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/01 09:25:19 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/01 09:25:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/01 09:25:19 INFO mlflow.sto

🧹 Limpiando y reconstruyendo dataset en: c:\Users\andre\OneDrive\Documents\UPS\Inteligencia Artificial\YOLOv8_living-room_furniture\data\processed_dataset
   ✅ train: 3530 imágenes procesadas con éxito.
   ✅ valid: 149 imágenes procesadas con éxito.

🚀 Iniciando entrenamiento del Baseline v1...
New https://pypi.org/project/ultralytics/8.4.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.8  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\andre\OneDrive\Documents\UPS\Inteligencia Artificial\YOLOv8_living-room_furniture\data\v1_fixed_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed

2026/02/01 09:27:02 INFO mlflow.tracking.fluent: Experiment with name 'c:\Users\andre\OneDrive\Documents\UPS\Inteligencia Artificial\YOLOv8_living-room_furniture\models_history' does not exist. Creating a new experiment.


MLflow: logging run_id(0dec3c2842974863af8f9da41f94e280) to sqlite:///c:/Users/andre/OneDrive/Documents/UPS/Inteligencia Artificial/YOLOv8_living-room_furniture/mlflow.db
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 480 train, 480 val
Using 8 dataloader workers
Logging results to C:\Users\andre\OneDrive\Documents\UPS\Inteligencia Artificial\YOLOv8_living-room_furniture\models_history\v1_run
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        1/5     0.637G      1.511       3.49       1.45        180        480: 100% ━━━━━━━━━━━━ 11/11 2.8it/s 3.9s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 4.3it/s 1.2s0.3s
                   all        149        735    0.00862      0.703      0.365      0.271

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        2/5     0.918G      1.316      2.611      1.288    